In [ ]:
from torch import nn
from torchvision import transforms
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

from ml_tools.ML_vision_datasetmaster import DragonDatasetVision
from ml_tools.ML_trainer import DragonTrainer
from ml_tools.ML_callbacks import DragonModelCheckpoint, DragonEarlyStopping, DragonLRScheduler
from ml_tools.ML_utilities import inspect_model_architecture
from ml_tools.ML_configuration import MultiClassImageClassificationMetricsFormat, FinalizeMultiClassImageClassification, DragonTrainingConfig
from ml_tools.IO_tools import train_logger

from rootpaths import PM
from visual_ccc.gradcam import custom_alexnet, SIZE_REQUIREMENT

In [ ]:
train_config = DragonTrainingConfig(validation_size=0.2,
                                    test_size=0.1,
                                    initial_learning_rate=0.0002,
                                    batch_size=2,
                                    early_stop_patience=30,
                                    scheduler_patience=4,
                                    scheduler_lr_factor=0.5)

## Multiclass classification: Dendrites, Spheroids, Alloys

In [ ]:
vision_dataset = DragonDatasetVision.from_folder(PM.three_dataset)

vision_dataset.split_data(val_size=train_config.validation_size, 
                          test_size=train_config.test_size,
                          random_state=train_config.random_state)

vision_dataset.configure_transforms(resize_size=SIZE_REQUIREMENT,
                                    crop_size=None,
                                    mean=None, std=None,
                                    pre_transforms=[transforms.Grayscale(num_output_channels=1)])

In [ ]:
class_map = vision_dataset.save_class_map(save_dir=PM.artifacts)

vision_dataset.save_transform_recipe(filepath=PM.transform_recipe_file)

In [ ]:
# Model
model = custom_alexnet()
original_in_features: int = model.classifier[6].in_features # type: ignore
model.classifier[6] = nn.Linear(in_features=original_in_features, out_features=3, bias=True) # 3 class classification

inspect_model_architecture(model=model, save_dir=PM.artifacts)

# Optimizer
optimizer = AdamW(params=model.parameters(), lr=train_config.initial_learning_rate)

# Trainer
trainer = DragonTrainer(model=model,
                    train_dataset=vision_dataset.train_dataset,
                    validation_dataset=vision_dataset.validation_dataset,
                    kind="multiclass image classification",
                    optimizer=optimizer,
                    criterion="auto",
                    device="cuda:0",
                    checkpoint_callback=DragonModelCheckpoint(save_dir=PM.checkpoints, mode="min"),
                    early_stopping_callback=DragonEarlyStopping(patience=train_config.early_stop_patience, mode="min"), # type: ignore
                    lr_scheduler_callback=DragonLRScheduler(scheduler=ReduceLROnPlateau(optimizer=optimizer, 
                                                                                        mode="min",
                                                                                        factor=train_config.scheduler_lr_factor, # type: ignore
                                                                                        patience=train_config.scheduler_patience)) # type: ignore
                    )

In [ ]:
history = trainer.fit(save_dir=PM.artifacts, epochs=200, batch_size=train_config.batch_size)

In [ ]:
train_logger(train_config=train_config,
             model_parameters={'architecture': 'custom Alexnet'},
             train_history=history,
             save_directory=PM.results)

In [ ]:
trainer.evaluate(save_dir=PM.metrics, 
                 model_checkpoint="latest",
                 test_data=vision_dataset.test_dataset,
                 val_format_configuration=MultiClassImageClassificationMetricsFormat(cmap='YlGn', ROC_PR_line="darkorange"),
                 test_format_configuration=MultiClassImageClassificationMetricsFormat(cmap='Oranges', ROC_PR_line="forestgreen")
                 )

In [ ]:
finalizer = FinalizeMultiClassImageClassification(filename="AlloysDendritesSpheroids", class_map=class_map)

trainer.finalize_model_training(model_checkpoint="current",
                                save_dir=PM.artifacts,
                                finalize_config=finalizer)